# TACO Trash Detector using Mask R-CNN

![](https://raw.githubusercontent.com/wiki/pedropro/TACO/images/teaser.gif)

## Repro [Github](https://github.com/pedropro/TACO)

In [1]:
!pip install pycocotools
!git clone https://github.com/pedropro/TACO
%cd TACO/detector

  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl size=273763 sha256=7626cf562455c7701b868492574e2931fa7ad85e5f41da3d4a005df3dc2aea16
  Stored in directory: /root/.cache/pip/wheels/bc/cf/1b/e95c99c5f9d1648be3f500ca55e7ce55f24818b0f48336adaf
Successfully built pycocotools
Cloning into 'TACO'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 251 (delta 13), reused 12 (delta 4), pack-reused 219
Receiving objects: 100% (251/251), 62.48 MiB | 33.03 MiB/s, done.
Resolving deltas: 100% (130/130), done.
/kaggle/working/TACO/detector


In [2]:
import os
import sys
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

In [3]:
ROOT_DIR = os.path.abspath("/kaggle/working/TACO/detector")

import utils
import visualize
from visualize import display_images
import model as modellib
from config import Config
from model import log

## Load Dataset

In [4]:
!cp -rf ../../../input/tacotrashdataset/data ..

In [5]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

%matplotlib inline

In [6]:
data_path = '../../../input/tacotrashdataset/data'
anns_file_path = data_path +'/'+ 'annotations.json'

# Read annotations
with open(anns_file_path, 'r') as f:
    anno = json.loads(f.read())

categories = anno['categories']
anns = anno['annotations']
imgs = anno['images']
num_cats = len(categories)
num_annotations = len(anns)
num_images = len(imgs)

# Load categories and super categories
cat_names = []
super_cat_names = []
super_cat_ids = {}
super_cat_last_name = ''
num_super_cats = 0
for cat_it in categories:
    cat_names.append(cat_it['name'])
    super_cat_name = cat_it['supercategory']
    # Adding new supercat
    if super_cat_name != super_cat_last_name:
        super_cat_names.append(super_cat_name)
        super_cat_ids[super_cat_name] = num_super_cats
        super_cat_last_name = super_cat_name
        num_super_cats += 1

print('Number of super categories:', num_super_cats)
print('Number of categories:', num_cats)
print('Number of annotations:', num_annotations)
print('Number of images:', num_images)

Number of super categories: 28
Number of categories: 60
Number of annotations: 4784
Number of images: 1500


## Mask-RCNN Configuration

In [10]:
class TacoTestConfig(Config):
    NAME = "taco"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.3
    NUM_CLASSES = num_cats
#     IMAGE_MAX_DIM = 1024
#     IMAGE_MIN_DIM = 1024
#     IMAGE_RESIZE_MODE = "square"
config = TacoTestConfig()

## Image Cropping

In [17]:
# read meta_df.csv
df = pd.read_csv("../../../input/tacotrashdataset/meta_df.csv")